## Arena: Untertitel-Analyse
Analyse der Untertiteldaten der SRF-Arena-Sendungen zwischen 2016 und 2019

In [3]:
### import libraries
import pandas as pd
import numpy as np
from datetime import datetime
import srt
import re
import os
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

### Untertitel bereinigen
.srt-Dateien bereinigen und gemäss div. Merkmalen (wie z.B. Farbkodierung) Zeilen des Untertitels einzelnen Personen oder Ereignissen zuweisen 

In [31]:
### remove html-tags from string
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>|\(.*?\)')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

### tag lines in subtitles (if possible)
def identify_subtitles(subtitle):
    speaker = ["Moderator"]
    mask_host = "91FFFF"
    mask_others = None
    identified_subtitles = []
    
    for entry in subtitle:
        for line in entry.content.splitlines():
            ### Get clean string
            clean_line = cleanhtml(line)
                    
                    
            ### Switch between
            #### 1. Speaker indicated by parentheses
            #### 2. Speaker highlighted by color (in a special case = moderator)
            #### 3. Comment
            #### 4. Everything else
            
            if line.find("(") != -1:
                speaker.append(line[line.find("(")+1:line.find(")")])
            elif (line[0] == "<"):
                if mask_host == line[13:19]:
                    speaker.append("Moderator")
                else:
                    speaker.append("Gast")
            elif line.find("*") != -1:
                speaker.append("<---> Kommentar <--->")
            else:
                if ((speaker[-1] == "Moderator") or (speaker[-1] == "<---> Kommentar <--->")):
                    speaker.append("Gast")
            identified_subtitles.append([speaker[-1], clean_line.lstrip()])
    return identified_subtitles

In [88]:
### define data directory and initialize variables
directory = "data/"
words_moderator = 0
words_rest = 0
comments = []

### loop over .srt-files
for filename in os.listdir(directory):
    if (("srt" in filename)):
        with open(directory+filename, "rb") as file:
            ### decode and parse data
            data = file.read().decode("iso-8859-1")
            subtitle_generator = srt.parse(data)
            subtitle = list(subtitle_generator)
            identified_subtitles = identify_subtitles(subtitle)
            
            ### handle 1. counter for host, 2. comments and 3. counter for rest
            for line in identified_subtitles:
                if line[0] == "Moderator":
                    words_moderator += len(tokenizer.tokenize(line[1]))
                elif line[0] == "<---> Kommentar <--->":
                    comments.append([filename, line[1]])
                else:
                    words_rest += len(tokenizer.tokenize(line[1]))

---

### Kommentare auslesen und bereinigen

In [89]:
### create dataframe for comments
comments = pd.DataFrame(comments, columns = ["file", "comment"])
comments["counter"] = 1
comments["date"] = comments["file"].apply(lambda x: x[6:14])

In [41]:
### look at existing values
comments.groupby(["date", "comment"]).count().reset_index()["comment"].unique()

array(['* Bedrohlicher Geigenton *', '* Er spricht Arabisch. *',
       '* Fröhliche türkische Musik *', '* Klingeln *', '* Lachen *',
       '* Muezzinruf *', '* Sie sprechen durcheinander. *',
       '* Signalton *', '* Applaus *', '* Andreas Glarner lacht. *',
       '* Sie sprechen durcheinander *', '* Spannungsgeladene Musik *',
       '* Das Publikum lacht. *', '* Filippo Lombardi ruft',
       '* Sie reden durcheinander. *', 'etwas Unverständliches. *',
       '* Ausruf von Mitleid *', '* Jubel *',
       '* Schweizer Nationalhymne *', '* Vereinzelter Applaus *',
       '* Verhaltener Applaus *', '* Lachen im Publikum *',
       '* Tilman Slembeck', 'sagt etwas Unverständliches. *',
       '* Tamara Funiciello lacht. *', '* Orgelmusik *',
       '* Lachen im Studio *', '* Sirene *', '* Christoph Blocher',
       '* Christoph Blocher murmelt etwas. *',
       'murmelt etwas Unverständliches. *', '* Lachen, Applaus *',
       '* Einzelner Applaus *', '* Gemütlicher Rapsong',
     

In [90]:
### clean values by grouping ###
#### 1. "durcheinander reden" ####
comments.loc[comments.comment == "* Sie sprechen durcheinander *", 'comment'] = "* Sie sprechen durcheinander. *"
comments.loc[comments.comment == "* Sie reden durcheinander. *", 'comment'] = "* Sie sprechen durcheinander. *"
comments.loc[comments.comment == "* Sie sprechen durcheinander.*", 'comment'] = "* Sie sprechen durcheinander. *"
comments.loc[comments.comment == '* Sie sprechen durcheinander. "', 'comment'] = "* Sie sprechen durcheinander. *"
comments.loc[comments.comment == '* Sie sprechen gleichzeitig. *', 'comment'] = "* Sie sprechen durcheinander. *"
comments.loc[comments.comment == '* Unverständlicher Zwischenruf *', 'comment'] = "* Sie sprechen durcheinander. *"
comments.loc[comments.comment == '* Protestrufe *', 'comment'] = "* Sie sprechen durcheinander. *"
comments.loc[comments.comment == '* Unverständlicher Einwurf *', 'comment'] = "* Sie sprechen durcheinander. *"
comments.loc[comments.comment == '* Unverständlicher Einwand *', 'comment'] = "* Sie sprechen durcheinander. *"


#### 2. "Heiterkeit" ####
comments.loc[comments.comment == '* Das Publikum lacht. *', 'comment'] = "* Lachen *"
comments.loc[comments.comment == '* Lachen im Publikum *', 'comment'] = "* Lachen *"
comments.loc[comments.comment == "* Lachen im Studio *", 'comment'] = "* Lachen *"
comments.loc[comments.comment == "* Andreas Glarner lacht. *", 'comment'] = "* Lachen *"
comments.loc[comments.comment == "* Jubel *", 'comment'] = "* Lachen *"
comments.loc[comments.comment == "* Tamara Funiciello lacht. *", 'comment'] = "* Lachen *"
comments.loc[comments.comment == "* Lachen, Applaus *", 'comment'] = "* Lachen *"
comments.loc[comments.comment == "* Lachen und Applaus *", 'comment'] = "* Lachen *"
comments.loc[comments.comment == "* Lachen und vereinzelter Applaus *", 'comment'] = "* Lachen *"
comments.loc[comments.comment == "* Allgemeines Schmunzeln *", 'comment'] = "* Lachen *"
comments.loc[comments.comment == "* Lautes Lachen im Publikum *", 'comment'] = "* Lachen *"
comments.loc[comments.comment == "* Leises Lachen *", 'comment'] = "* Lachen *"
comments.loc[comments.comment == "* Lachen aus dem Publikum *", 'comment'] = "* Lachen *"
comments.loc[comments.comment == "* Gelächter *", 'comment'] = "* Lachen *"
comments.loc[comments.comment == "* Applaus und Jubelrufe *", 'comment'] = "* Lachen *"

In [91]:
### count different comments by date
comment_counter = comments[["date", "comment", "counter"]].groupby(["date", "comment"]).count().reset_index().sort_values(by="counter", ascending=False)

---

### Titel der Arena-Sendungen ermitteln
Sendungen nach Datum suchen und über https://www.srf.ch/sendungen/arena Titel, Beschreibungen und Gäste der Sendungen ermitteln

In [77]:
### import libraries
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from pathlib import Path
import json
import requests
import time
import re

In [78]:
### replace certain (wrong) dates
def replace_date(date):
    result = {
        "20160924" : "20160923",
        "20160531" : "20160527"
    }
    if date in result.keys():
        return(result[date])
    else:
        return(date)

In [92]:
### initialize variables
sendungen = []

### start browser and visit srf.ch
driver = webdriver.Chrome("./tools/chromedriver")
driver.get("https://www.srf.ch/sendungen/arena")

### reveal all entries back to 2016
for i in range(15):
    driver.find_element_by_class_name("btn-reveal").click()
    time.sleep(2)
    i += 1

### find entry by date and save title, description and link
for date in comment_counter.date.unique():
    date_check = replace_date(date)
    if (datetime.strptime(date_check, "%Y%m%d") > datetime.strptime("20190208", "%Y%m%d")):
        a_tag = (driver.find_element_by_xpath("//*[contains(text(), '{}')]/preceding-sibling::h3/a".format(datetime.strptime(date_check, "%Y%m%d").strftime("%-d.%-m.%Y"))))
        title = a_tag.text
        link = a_tag.get_attribute('href')
        description = (driver.find_element_by_xpath("//*[contains(text(), '{}')]/following-sibling::p".format(datetime.strptime(date_check, "%Y%m%d").strftime("%-d.%-m.%Y"))).text)
    else:
        a_tag = (driver.find_element_by_xpath("//*[contains(text(), '{}')]/preceding-sibling::h3/a".format(datetime.strptime(date_check, "%Y%m%d").strftime("%d.%m.%Y"))))
        title = a_tag.text
        link = a_tag.get_attribute('href')
        description = (driver.find_element_by_xpath("//*[contains(text(), '{}')]/following-sibling::p".format(datetime.strptime(date_check, "%Y%m%d").strftime("%d.%m.%Y"))).text)
    sendungen.append([date, title, description, link])


In [93]:
### create new dataframe and save data to csv
sendungen = pd.DataFrame(sendungen, columns = ["date", "title", "description", "link"])
sendungen.to_csv("data/sendungen.csv")

In [244]:
### join data (comments and shows)
comment_counter = pd.merge(comment_counter,
                           sendungen,
                           left_on="date",
                           right_on="date",
                           how="left")

In [320]:
### Initialize variable
guests = []
### Start browser
driver = webdriver.Chrome("./tools/chromedriver")
### visit each webpage of each show. then find and save guests.
for idx, row in sendungen.iterrows():
    driver.get(row.link)
    time.sleep(2)
    text_element = driver.find_element_by_xpath("/html[1]/body[1]/div[1]/div[2]/div[1]/div[1]/article[1]/div[1]/div[3]")
    list_elements = text_element.find_elements_by_tag_name("li")
    for elem in list_elements:
        full_desc = (elem.text).split(",")
        if len(full_desc) > 1:
            name = full_desc[0]
            title = full_desc[1]
        else:
            name = ""
            title = elem.text
        guests.append([row.date, name, title])
driver.close()
guests = pd.DataFrame(guests, columns=["date", "name", "title"])

In [321]:
### clean data
guests = guests[guests.name != ""]
guests.loc[guests.date == "20181116", 'title'] = guests[guests.date == "20181116"].name.apply(lambda x: x[x.find("(")+1:x.find(")")])
guests.loc[guests.date == "20181116", 'name'] = guests[guests.date == "20181116"].name.apply(lambda x: x[:x.find("(")-1])
guests = guests[guests.name != "Sollte der Finanzausgleich zwischen den Kantonen abgeschafft werden"]
guests = guests[~guests.title.str.contains("die wir selber vielleicht gar nicht brauchen?")]

In [330]:
### save data as csv
guests.to_csv("data/gaeste_nach_sendung.csv")
comment_counter.to_csv("data/kommentare.csv")

---

### Auswertungen

In [108]:
### look at most "chaotic" shows
pd.concat([comment_counter[comment_counter.comment == "* Sie sprechen durcheinander. *"], sendungen], sort=False).sort_values(by="counter", ascending=False)

,date,comment,counter,title,description,link
0,20181123,* Sie sprechen durcheinander. *,31.0,Gekaufte Abstimmungen?,Befürworter wie auch Gegner der Selbstbestimmu...,https://www.srf.ch/sendungen/arena/gekaufte-ab...
2,20161118,* Sie sprechen durcheinander. *,21.0,Kapitalismus oder Klassenkampf?,"Trump gewinnt, Europa rutscht nach rechts. Abe...",https://www.srf.ch/sendungen/arena/kapitalismu...
1,20170929,* Sie sprechen durcheinander. *,21.0,Am runden Tisch,Ob Unternehmenssteuer oder Altersvorsorge: In ...,https://www.srf.ch/sendungen/arena/am-runden-t...
3,20180309,* Sie sprechen durcheinander. *,20.0,Blocher – auf zum letzten Gefecht!,Nun geht es Schlag auf Schlag: Am Montag präse...,https://www.srf.ch/sendungen/arena/blocher-auf...
4,20170526,* Sie sprechen durcheinander. *,20.0,Die Parteipräsidenten,Fast ist Halbzeit in Bern: Was hat die erste L...,https://www.srf.ch/sendungen/arena/die-parteip...
5,20171117,* Sie sprechen durcheinander. *,19.0,Profit statt Moral?,Steuervermeidung und Rohstoffhandel: Von beide...,https://www.srf.ch/sendungen/arena/profit-stat...
6,20170922,* Sie sprechen durcheinander. *,17.0,Blocher gegen alle,Ob der neue Aussenminister nun Ignazio Cassis ...,https://www.srf.ch/sendungen/arena/blocher-geg...
7,20161104,* Sie sprechen durcheinander. *,17.0,Wer hat das Sagen im Land?,Unterdessen ist klar: Die Masseneinwanderungsi...,https://www.srf.ch/sendungen/arena/wer-hat-das...
8,20170407,* Sie sprechen durcheinander. *,17.0,Die Attacke,Die SVP zielt auf die Menschenrechts-Konventio...,https://www.srf.ch/sendungen/arena/die-attacke
9,20170505,* Sie sprechen durcheinander. *,15.0,Sind wir zu freizügig?,Europa wird immer EU-kritischer: In Frankreich...,https://www.srf.ch/sendungen/arena/sind-wir-zu...


In [106]:
### look at most "funny" shows
pd.concat([comment_counter[comment_counter.comment == "* Lachen *"], sendungen], sort=False).sort_values(by="counter", ascending=False)

,date,comment,counter,title,description,link
12,20170519,* Lachen *,13.0,Eine tierische «Arena»,Am sechsten Tag schuf Gott Mensch und Landtier...,https://www.srf.ch/sendungen/arena/eine-tieris...
24,20161111,* Lachen *,8.0,Wieso Trump?,Wie war diese Wahl möglich? Diese Frage stelle...,https://www.srf.ch/sendungen/arena/wieso-trump
29,20170922,* Lachen *,7.0,Blocher gegen alle,Ob der neue Aussenminister nun Ignazio Cassis ...,https://www.srf.ch/sendungen/arena/blocher-geg...
42,20180622,* Lachen *,6.0,SRG und Private – wer profitiert vom neuen Med...,Wie weiter mit den Schweizer Medien? Drei Mona...,https://www.srf.ch/sendungen/arena/srg-und-pri...
57,20180105,* Lachen *,4.0,No Billag - jetzt redet das Volk!,Braucht es Radio- und TV-Gebühren? Das ist die...,https://www.srf.ch/sendungen/arena/no-billag-j...
66,20181102,* Lachen *,4.0,«Abstimmungs-Arena»: Hornkuh-Initiative,Mehr Kühe mit Hörnern – das will die Hornkuh-I...,https://www.srf.ch/sendungen/arena/abstimmungs...
89,20171006,* Lachen *,3.0,Burka – da sehen wir schwarz!,Die Burka gibt zu reden. In Österreich ist sie...,https://www.srf.ch/sendungen/arena/burka-da-se...
96,20180119,* Lachen *,3.0,«Abstimmungsarena»: Finanzordnung 2021,"Darf der Bund weiterhin Steuern erheben? Ja, s...",https://www.srf.ch/sendungen/arena/abstimmungs...
95,20160617,* Lachen *,3.0,Gut genug für den Schweizer Pass?,Die Schweiz hat europaweit eines der strengste...,https://www.srf.ch/sendungen/arena/gut-genug-f...
93,20161118,* Lachen *,3.0,Kapitalismus oder Klassenkampf?,"Trump gewinnt, Europa rutscht nach rechts. Abe...",https://www.srf.ch/sendungen/arena/kapitalismu...


In [333]:
### determine number of appearances by guest
guests.name.value_counts()

Petra Gössi                      10
Albert Rösti                     8 
Regula Rytz                      8 
Ruth Humbel                      7 
Alain Berset                     6 
Christoph Blocher                6 
Rosmarie Quadranti               5 
Christa Tobler                   5 
Tiana Angelina Moser             5 
Elisabeth Schneider-Schneiter    5 
Balthasar Glättli                5 
Gerhard Pfister                  4 
Eric Nussbaumer                  4 
Ruedi Noser                      4 
Tamara Funiciello                4 
Pirmin Bischof                   4 
Cédric Wermuth                   3 
Doris Leuthard                   3 
Martin Landolt                   3 
Paul Rechsteiner                 3 
Markus Ritter                    3 
Christian Levrat                 3 
Claudio Zanetti                  3 
Andreas Glarner                  3 
Valentin Vogt                    3 
Regine Sauter                    3 
Silvan Amberg                    3 
Christine Häsler            